# Poisson in 1D with scikit-fem

We solve the boundary value problem
$$-u''(x)=1,\quad x\in(0,1),\qquad u(0)=u(1)=0.$$

**Exact solution:** $u(x)=\tfrac{1}{2}x(1-x)$.
We use a P1 (linear) finite element discretization on a uniform 1D mesh.

## Step 1: Import Required Libraries

We start by importing the necessary libraries:
- `numpy` for numerical operations
- `matplotlib` for plotting
- `skfem` components for finite element computations
- `skfem.helpers` provides decorators and functions for defining variational forms

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skfem import MeshLine, Basis, ElementLineP1, asm
from skfem.helpers import *

## Step 2: Define Mesh and Function Space

Here we create the computational domain and finite element space:
- `MeshLine` creates a 1D mesh on the interval [0,1]
- `ElementLineP1` defines linear (P1) finite elements
- `Basis` combines the mesh and element to create the function space

The refinement level controls mesh resolution: `refine=6` gives 2^6=64 elements.

In [ ]:
# Mesh and basis on [0, 1]
refine = 6   # 64 elements
m = MeshLine(np.linspace(0, 1, 2**refine + 1))
e = ElementLineP1()
basis = Basis(m, e)

## Step 3: Define Weak Form (Variational Formulation)

The weak form of our PDE $-u'' = 1$ is: Find $u$ such that
$$\int_0^1 u' v' \, dx = \int_0^1 1 \cdot v \, dx \quad \forall v$$

We define:
- **Bilinear form** (LHS): $a(u,v) = \int \nabla u \cdot \nabla v \, dx$
- **Linear form** (RHS): $L(v) = \int f \cdot v \, dx$ where $f=1$

The `@BilinearForm` and `@LinearForm` decorators create functions that can be assembled into matrices.

In [ ]:
# Define bilinear form: ∫ ∇u · ∇v dx
@BilinearForm
def laplace_form(u, v, _):
    return u.grad[0] * v.grad[0]  # In 1D, grad is just du/dx

# Define linear form: ∫ f * v dx (where f = 1)
@LinearForm  
def load_form(v, _):
    return 1.0 * v

## Step 4: Assembly

The `asm()` function assembles the variational forms into:
- **Stiffness matrix** `A`: represents the bilinear form
- **Load vector** `b`: represents the linear form

This creates the linear system $Au = b$ that approximates our PDE.

In [ ]:
# Assemble stiffness and load for -u'' = 1
A = asm(laplace_form, basis)
b = asm(load_form, basis)

print(f"System size: {A.shape[0]} x {A.shape[1]}")
print(f"Matrix type: {type(A)}")

## Step 5: Apply Boundary Conditions and Solve

We enforce Dirichlet boundary conditions $u(0) = u(1) = 0$ by:
1. Finding boundary nodes (at x=0 and x=1)
2. Modifying the system: set boundary rows to identity, RHS to zero
3. Converting sparse matrix to dense for `np.linalg.solve()`
4. Solving the linear system

This is called the "elimination method" for applying essential boundary conditions.

In [ ]:
# Find boundary nodes (x=0 and x=1)
x = m.p[0]
boundary = np.where((np.abs(x) < 1e-12) | (np.abs(x - 1.0) < 1e-12))[0]
print(f"Boundary nodes: {boundary} (at x = {x[boundary]})")

# Apply Dirichlet BCs: convert to dense and modify
A_dense = A.toarray()
for i in boundary:
    A_dense[i, :] = 0
    A_dense[i, i] = 1
    b[i] = 0

# Solve the system
u = np.linalg.solve(A_dense, b)
print(f"Solution computed with {len(u)} degrees of freedom")

## Step 6: Validation and Visualization

We compare our numerical solution with the analytical solution $u_{exact}(x) = \frac{1}{2}x(1-x)$.

The error analysis helps us understand:
- How accurate our approximation is
- How the error decreases with mesh refinement (convergence)

For P1 elements, we expect $O(h^2)$ convergence in the $L^\infty$ norm.

In [ ]:
# Exact solution and error analysis
u_exact = 0.5 * x * (1 - x)
err_inf = np.max(np.abs(u - u_exact))
h = 1.0 / (2**refine)  # mesh size
print(f"Mesh size h: {h:.6f}")
print(f"Infinity-norm error: {err_inf:.3e}")
print(f"Error/h^2 ratio: {err_inf/h**2:.3f} (should be roughly constant for P1)")

# Plot FE vs exact solution
plt.figure(figsize=(10,6))

plt.subplot(1,2,1)
plt.plot(x, u, marker='o', markersize=3, label='FE (P1)', linewidth=2)
plt.plot(x, u_exact, linestyle='--', label='Exact: 0.5·x·(1-x)', linewidth=2)
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('Solution Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1,2,2)
plt.plot(x, np.abs(u - u_exact), 'r-', linewidth=2)
plt.xlabel('x')
plt.ylabel('|error|')
plt.title('Pointwise Error')
plt.yscale('log')
plt.grid(True, alpha=0.3)

plt.suptitle('1D Poisson: -u"=1 on (0,1), u(0)=u(1)=0', fontsize=14)
plt.tight_layout()
plt.show()

### Experiments
- Change `refine` and observe the error decrease.
- Replace the RHS with a function by defining a custom load form.
- Try nonzero Dirichlet data by modifying the condensation step.